# Feature Transformation with Amazon a SageMaker Processing Job and Apache Spark

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Apache Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Apache Spark in a managed SageMaker environment to run our processing workload.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)


# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [1]:
import sagemaker
from time import gmtime, strftime
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Setup Input Data

In [2]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-west-2-140773038493/amazon-reviews-pds/tsv/


In [3]:
!aws s3 ls $s3_input_data

2020-07-25 17:24:19   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-07-25 17:24:22   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


# Build a Spark Docker Image to Run the Processing Job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset processing.

Build the example Spark container.

In [4]:
docker_repo = 'amazon-reviews-spark-processor'
docker_tag = 'latest'

In [5]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  4.441MB
Step 1/37 : FROM openjdk:8-jre-slim
 ---> f2e91f81bf2c
Step 2/37 : RUN apt-get update
 ---> Using cache
 ---> 62231d826b5c
Step 3/37 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> bce0d33778c9
Step 4/37 : RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
 ---> Using cache
 ---> aefbffba7dcd
Step 5/37 : RUN apt-get clean
 ---> Using cache
 ---> b2df2b55de1c
Step 6/37 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 89a06781e68c
Step 7/37 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> e4cfc81c854b
Step 8/37 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 6fb94f0f28c4
Step 9/37 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> 39444b1d9efa
Step 10/37 : ENV HADOOP_VERSION 3.2.1
 ---> Using cache
 ---> 0d511b3986f9
Step 11/37 : ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
 ---> Using cache
 ---> 1117bac7ad00
Step 12/37 : ENV HADOOP

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [6]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

140773038493.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor:latest


### Create ECR repository and push docker image

In [7]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


### Ignore any `RepositoryNotFoundException` error, we are creating the repo right after.

In [8]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo


An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'amazon-reviews-spark-processor' does not exist in the registry with id '140773038493'
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:140773038493:repository/amazon-reviews-spark-processor",
        "registryId": "140773038493",
        "repositoryName": "amazon-reviews-spark-processor",
        "repositoryUri": "140773038493.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor",
        "createdAt": 1595702488.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        }
    }
}


In [9]:
!docker tag $docker_repo:$docker_tag $image_uri

In [10]:
!docker push $image_uri

The push refers to repository [140773038493.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor]

dcf6e5d7: Preparing 
2184851c: Preparing 
b4dce864: Preparing 
d67bce84: Preparing 
0c69a1da: Preparing 
945badba: Preparing 
f6318e65: Preparing 
68ba4e24: Preparing 
5db8da73: Preparing 
484ea491: Preparing 
882873f1: Preparing 
ef63f751: Preparing 
09cb497b: Preparing 
5db8da73: Waiting g 
4809f1d0: Preparing 
c95dcfbb: Preparing 
38d128fe: Preparing 
e96fc851: Waiting g 


cf6e5d7: Pushing  1.794GB/2.011GB

cf6e5d7: Pushed   2.021GB/2.011GBlatest: digest: sha256:3335980bdfcab313fe973147d70f4bd0994355faa82a378e5f26b336b07b4e71 size: 4318


# Run the Job using Amazon SageMaker Processing Jobs

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a Spark ML script for processing in the job configuration.

Review the Spark processing script.

In [11]:
!pygmentize preprocess-spark-text-to-bert.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv
import collections
import subprocess
import sys
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pip', '--upgrade'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'wrapt', '--upgrade', '--ignore-installed'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0', '--ignore-installed'])
import tensorflow as tf
print(tf.__version__)
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
from transformers import DistilBertTokenizer

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import split
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

tokenizer = DistilBertTokenizer.from_pretrained('distil

In [12]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-processor',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=2, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.xlarge',
                            env={'mode': 'python'})

# Setup Output Data

In [13]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)

In [14]:
train_data_bert_output = 's3://{}/{}/output/bert-train'.format(bucket, output_prefix)
validation_data_bert_output = 's3://{}/{}/output/bert-validation'.format(bucket, output_prefix)
test_data_bert_output = 's3://{}/{}/output/bert-test'.format(bucket, output_prefix)

print(train_data_bert_output)
print(validation_data_bert_output)
print(test_data_bert_output)

s3://sagemaker-us-west-2-140773038493/amazon-reviews-spark-processor-2020-07-25-18-42-56/output/bert-train
s3://sagemaker-us-west-2-140773038493/amazon-reviews-spark-processor-2020-07-25-18-42-56/output/bert-validation
s3://sagemaker-us-west-2-140773038493/amazon-reviews-spark-processor-2020-07-25-18-42-56/output/bert-test


In [15]:
from sagemaker.processing import ProcessingOutput

processor.run(code='preprocess-spark-text-to-bert.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_train_data', train_data_bert_output,
                         's3_output_validation_data', validation_data_bert_output,
                         's3_output_test_data', test_data_bert_output,                         
              ],
              # We need this dummy output to allow us to call 
              #    ProcessingJob.from_processing_name() later 
              #    to describe the job and poll for Completed status                            
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-train',
                                        source='/opt/ml/processing/output/bert/train'),
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-validation',
                                        source='/opt/ml/processing/output/bert/validation'),
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-test',
                                        source='/opt/ml/processing/output/bert/test'),
              ],          
              logs=True,
              wait=False
)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  spark-amazon-reviews-processor-2020-07-25-18-42-56-429
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/spark-amazon-reviews-processor-2020-07-25-18-42-56-429/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'bert-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/spark-amazon-reviews-processor-2020-07-25-18-42-56-429/output/bert-train', 'LocalPath': '/opt/ml/processing/output/bert/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-validation', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/spark-amazon-reviews-processor-2020-07-25-18-42-56-429/output/bert-validation', 'LocalPath': '/opt/ml/processing/output/bert/validation', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-test', 'S3Output': {'S3Uri

In [16]:
from IPython.core.display import display, HTML

spark_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, spark_processing_job_name)))


In [17]:
from IPython.core.display import display, HTML

# This is different than the job name because we are not using ProcessingOutput's in this Spark ML case.
spark_processing_job_s3_output_prefix = output_prefix

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(bucket, spark_processing_job_s3_output_prefix, region)))


# List Processing Jobs through boto3 Python SDK

In [18]:
import boto3

client = boto3.client('sagemaker')
client.list_processing_jobs()

{'ProcessingJobSummaries': [{'ProcessingJobName': 'spark-amazon-reviews-processor-2020-07-25-18-42-56-429',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-west-2:140773038493:processing-job/spark-amazon-reviews-processor-2020-07-25-18-42-56-429',
   'CreationTime': datetime.datetime(2020, 7, 25, 18, 42, 56, 878000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 7, 25, 18, 42, 56, 878000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'InProgress'},
  {'ProcessingJobName': 'sagemaker-scikit-learn-2020-07-25-18-35-13-587',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-west-2:140773038493:processing-job/sagemaker-scikit-learn-2020-07-25-18-35-13-587',
   'CreationTime': datetime.datetime(2020, 7, 25, 18, 35, 14, 34000, tzinfo=tzlocal()),
   'ProcessingEndTime': datetime.datetime(2020, 7, 25, 18, 39, 29, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 7, 25, 18, 39, 29, 281000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'Completed'},
  {'ProcessingJobName'

# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [19]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=spark_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/spark-amazon-reviews-processor-2020-07-25-18-42-56-429/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'bert-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/spark-amazon-reviews-processor-2020-07-25-18-42-56-429/output/bert-train', 'LocalPath': '/opt/ml/processing/output/bert/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-validation', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/spark-amazon-reviews-processor-2020-07-25-18-42-56-429/output/bert-validation', 'LocalPath': '/opt/ml/processing/output/bert/validation', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-test', 'S3Output': {'S3Uri': 's3://sa

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

In [20]:
running_processor.wait()

2020-07-25 18:46:25,459 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = algo-1/10.0.154.221
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.2.1
STARTUP_MSG:   classpath = /usr/hadoop-3.2.1/etc/hadoop:/usr/hadoop-3.2.1/share/hadoop/common/lib/kerb-core-1.0.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/kerb-admin-1.0.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/kerb-common-1.0.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jackson-core-2.9.8.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jetty-util-9.3.24.v20180605.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/commons-collections-3.2.2.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jackson-mapper-asl-1.9.13.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jaxb-api-2.2.11.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jackson-annotations-2.9.8.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/kerby-asn1-1.0.1.jar:/usr/ha

Starting resourcemanager
Starting nodemanagers
localhost: /usr/hadoop-3.2.1/bin/../libexec/hadoop-functions.sh: line 982: ssh: command not found
2020-07-25 18:46:38,901 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-07-25 18:46:39.856544: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-07-25 18:46:39.856643: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-07-25 18:46:39.856653: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are in

2020-07-25 18:46:53,352 INFO ui.JettyUtils: Adding filter org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter to /metrics/json.
2020-07-25 18:46:53,359 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@1f22c55e{/metrics/json,null,AVAILABLE,@Spark}
2020-07-25 18:46:55,838 INFO cluster.YarnSchedulerBackend$YarnDriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (10.0.190.110:58510) with ID 1
2020-07-25 18:46:55,925 INFO storage.BlockManagerMasterEndpoint: Registering block manager algo-2:35743 with 11.9 GB RAM, BlockManagerId(1, algo-2, 35743, None)
2020-07-25 18:47:12,506 INFO cluster.YarnClientSchedulerBackend: SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2020-07-25 18:47:12,734 INFO internal.SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.4.6/spark-warehouse').
2020-07-25 18:47:12,734 INFO internal.

2020-07-25 18:47:23,318 INFO storage.BlockManagerInfo: Added broadcast_7_piece0 in memory on algo-2:35743 (size: 42.9 KB, free: 11.9 GB)
2020-07-25 18:47:23,424 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 4.0 (TID 4) in 158 ms on algo-2 (executor 1) (1/1)
2020-07-25 18:47:23,425 INFO cluster.YarnScheduler: Removed TaskSet 4.0, whose tasks have all completed, from pool 
2020-07-25 18:47:23,425 INFO scheduler.DAGScheduler: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0) finished in 0.165 s
2020-07-25 18:47:23,426 INFO scheduler.DAGScheduler: Job 4 finished: showString at NativeMethodAccessorImpl.java:0, took 0.168150 s
+-----------+--------------------+
|star_rating|         review_body|
+-----------+--------------------+
|          2|I keep buying mad...|
|          5|             Awesome|
|          5|If you are preppi...|
|          5|             Perfect|
|          5|            Awesome!|
|          5|            Awesome!|
|          4|i like the new sk..

2020-07-25 18:57:04,484 INFO scheduler.TaskSetManager: Finished task 1.0 in stage 7.0 (TID 8) in 575051 ms on algo-2 (executor 1) (1/2)
2020-07-25 19:00:38,947 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 7.0 (TID 7) in 789515 ms on algo-2 (executor 1) (2/2)
2020-07-25 19:00:38,947 INFO cluster.YarnScheduler: Removed TaskSet 7.0, whose tasks have all completed, from pool 
2020-07-25 19:00:38,948 INFO scheduler.DAGScheduler: ResultStage 7 (runJob at SparkHadoopWriter.scala:78) finished in 789.548 s
2020-07-25 19:00:38,948 INFO scheduler.DAGScheduler: Job 7 finished: runJob at SparkHadoopWriter.scala:78, took 789.551438 s
2020-07-25 19:00:38,968 INFO io.SparkHadoopWriter: Job job_20200725184729_0038 committed.
Wrote to output file:  /opt/ml/processing/output/bert/train
2020-07-25 19:00:39,012 INFO datasources.FileSourceStrategy: Pruning directories with: 
2020-07-25 19:00:39,012 INFO datasources.FileSourceStrategy: Post-Scan Filters: AtLeastNNulls(n, review_body#13)
2020-07-

2020-07-25 19:23:11,972 INFO scheduler.TaskSetManager: Finished task 1.0 in stage 9.0 (TID 12) in 569098 ms on algo-2 (executor 1) (1/2)
2020-07-25 19:26:43,625 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 9.0 (TID 11) in 780753 ms on algo-2 (executor 1) (2/2)
2020-07-25 19:26:43,625 INFO cluster.YarnScheduler: Removed TaskSet 9.0, whose tasks have all completed, from pool 
2020-07-25 19:26:43,626 INFO scheduler.DAGScheduler: ResultStage 9 (runJob at SparkHadoopWriter.scala:78) finished in 780.787 s
2020-07-25 19:26:43,627 INFO scheduler.DAGScheduler: Job 9 finished: runJob at SparkHadoopWriter.scala:78, took 780.789945 s
2020-07-25 19:26:43,642 INFO io.SparkHadoopWriter: Job job_20200725191342_0060 committed.
Wrote to output file:  /opt/ml/processing/output/bert/test
2020-07-25 19:26:43,661 INFO memory.MemoryStore: Block broadcast_19 stored as values in memory (estimated size 389.7 KB, free 363.9 MB)
2020-07-25 19:26:43,678 INFO memory.MemoryStore: Block broadcast_19_piec

2020-07-25 19:26:46
Finished Yarn configuration files setup.

Received end of job signal, exiting...
Finished Yarn configuration files setup.




# Inspect the Processed Output Dataset


## _The next cells will not work properly until the job completes above._


Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [21]:
!aws s3 ls --recursive $train_data_bert_output/

In [22]:
!aws s3 ls --recursive $validation_data_bert_output/

In [23]:
!aws s3 ls --recursive $test_data_bert_output/

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();